In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from ib_insync import *
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.stats import norm
from scipy.optimize import brentq

# ---- Parameters ----
symbol = 'GOOG'
expiry = '20251219'
strike = 195
right = 'C'  # 'C' for call, 'P' for put
durationStr = '7 D'
barSizeSetting = '5 mins'  # '1 min' for finer granularity but may hit IBKR limits
r = 0.044  # Risk-free rate (annual decimal)

# --- Black-Scholes helper functions ---

def d1(S, K, r, sigma, T):
    return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))


def d2(S, K, r, sigma, T):
    return d1(S, K, r, sigma, T) - sigma * np.sqrt(T)


def bs_price(S, K, r, sigma, T, option_type):
    D1 = d1(S, K, r, sigma, T)
    D2 = d2(S, K, r, sigma, T)
    if option_type == 'C':
        return S * norm.cdf(D1) - K * np.exp(-r * T) * norm.cdf(D2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-D2) - S * norm.cdf(-D1)


def bs_delta(S, K, r, sigma, T, option_type):
    D1 = d1(S, K, r, sigma, T)
    if option_type == 'C':
        return norm.cdf(D1)
    else:
        return norm.cdf(D1) - 1


def bs_gamma(S, K, r, sigma, T):
    D1 = d1(S, K, r, sigma, T)
    return norm.pdf(D1) / (S * sigma * np.sqrt(T))


def bs_vega(S, K, r, sigma, T):
    D1 = d1(S, K, r, sigma, T)
    return S * norm.pdf(D1) * np.sqrt(T) / 100  # per 1% vol change


def bs_theta(S, K, r, sigma, T, option_type):
    D1 = d1(S, K, r, sigma, T)
    D2 = d2(S, K, r, sigma, T)
    pdf_D1 = norm.pdf(D1)
    if option_type == 'C':
        theta = (
            -S * pdf_D1 * sigma / (2 * np.sqrt(T))
            - r * K * np.exp(-r * T) * norm.cdf(D2)
        )
    else:
        theta = (
            -S * pdf_D1 * sigma / (2 * np.sqrt(T))
            + r * K * np.exp(-r * T) * norm.cdf(-D2)
        )
    return theta / 365  # per calendar day


def implied_volatility(market_price, S, K, r, T, option_type):
    def objective(sigma):
        return bs_price(S, K, r, sigma, T, option_type) - market_price
    try:
        return brentq(objective, 1e-4, 5.0, maxiter=500)
    except Exception:
        return np.nan

# ---- Data Fetch ----

async def get_option_stock_data(symbol, expiry, strike, right, duration, barsize):
    ib = IB()
    await ib.connectAsync('127.0.0.1', 7497, clientId=np.random.randint(1000, 9999))

    stock = Stock(symbol, 'SMART', 'USD')
    opt = Option(symbol, expiry, strike, right, 'SMART')

    await ib.qualifyContractsAsync(stock)
    await ib.qualifyContractsAsync(opt)

    stock_bars = await ib.reqHistoricalDataAsync(
        stock, endDateTime='', durationStr=duration, barSizeSetting=barsize,
        whatToShow='TRADES', useRTH=True, formatDate=1
    )
    df_stock = util.df(stock_bars)
    df_stock.set_index('date', inplace=True)
    df_stock.rename(columns={'close': 'Stock Price'}, inplace=True)

    opt_bars = await ib.reqHistoricalDataAsync(
        opt, endDateTime='', durationStr=duration, barSizeSetting=barsize,
        whatToShow='TRADES', useRTH=True, formatDate=1
    )
    df_opt = util.df(opt_bars)
    df_opt.set_index('date', inplace=True)
    df_opt.rename(columns={'close': 'Option Close'}, inplace=True)

    ib.disconnect()

    result = df_stock[['Stock Price']].join(df_opt[['Option Close']], how='inner')
    result.dropna(subset=['Stock Price', 'Option Close'], inplace=True)
    return result

# ---- Main Async Code ----

async def main():
    df = await get_option_stock_data(
        symbol, expiry, strike, right, durationStr, barSizeSetting
    )
    return df

df = asyncio.run(main())

expiry_date = pd.to_datetime(expiry, format='%Y%m%d')
df.index = df.index.tz_localize(None)
df['T'] = (expiry_date - df.index).total_seconds() / (365.25 * 24 * 3600)
df = df[df['T'] > 0].copy()

# --- Calculate Greeks ---

def calculate_greeks(row):
    S = row['Stock Price']
    K = strike
    price = row['Option Close']
    T = row['T']
    option_type = right

    iv = implied_volatility(price, S, K, r, T, option_type)
    if np.isnan(iv):
        return pd.Series([np.nan]*5)
    delta = bs_delta(S, K, r, iv, T, option_type)
    gamma = bs_gamma(S, K, r, iv, T)
    vega = bs_vega(S, K, r, iv, T)
    theta = bs_theta(S, K, r, iv, T, option_type)
    return pd.Series([iv, delta, gamma, vega, theta])

df[['IV_calculated', 'Delta', 'Gamma', 'Vega', 'Theta']] = df.apply(calculate_greeks, axis=1)

from dash import Dash, dcc, html, Input, Output, State

app = Dash(__name__)

columns = [
    'Stock Price', 'Option Close',
    'IV_calculated', 'Delta', 'Gamma', 'Vega', 'Theta'
]
y_names = {
    'Stock Price': 'Price ($)',
    'Option Close': 'Price ($)',
    'IV_calculated': 'Implied Volatility (%)',
    'Delta': 'Delta',
    'Gamma': 'Gamma',
    'Vega': 'Vega',
    'Theta': 'Theta'
}

app.layout = html.Div([
    dcc.Interval(id='interval-component', interval=1000, n_intervals=0),
    *[
        dcc.Graph(id=f'graph-{col}') for col in columns
    ]
])

@app.callback(
    [Output(f'graph-{col}', 'figure') for col in columns],
    Input('interval-component', 'n_intervals'),
    [State(f'graph-{col}', 'relayoutData') for col in columns]
)
def update_graphs(n_intervals, *relayouts):
    figures = []
    xrange = None
    for relayout in relayouts:
        if relayout and 'xaxis.range[0]' in relayout and 'xaxis.range[1]' in relayout:
            xrange = [relayout['xaxis.range[0]'], relayout['xaxis.range[1]']]
            break

    for col in columns:
        fig = go.Figure()
        if col == 'IV_calculated':
            yvals = 100 * df[col]  # Plot IV as a percentage
            y_axis_title = "Implied Volatility (%)"
            trace_name = "IV (%)"
        else:
            yvals = df[col]
            y_axis_title = y_names[col]
            trace_name = col

        fig.add_trace(go.Scatter(
            x=df.index, y=yvals,
            mode='lines',
            name=trace_name,
            line=dict(width=2)
        ))
        layout_kwargs = dict(
            title=f"{col} over Time",
            xaxis_title='Date/Time',
            yaxis_title=y_axis_title,
            template='plotly_white',
            width=900,
            height=500,
            hovermode='x unified'
        )
        if xrange:
            layout_kwargs['xaxis_range'] = xrange

        fig.update_layout(**layout_kwargs)
        figures.append(fig)
    return figures

if __name__ == '__main__':
    app.run(debug=True)
